<a href="https://colab.research.google.com/github/Bolorooo24/CV-ML_labs/blob/main/Pytorch-deep-learning/03_pytorch_computer_vision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pytorch computer vision
* See the reference notebook : https://github.com/mrdbourke/pytorch-deep-learning/blob/main/video_notebooks/03_pytorch_computer_vision_video.ipynb
* See reference online book : https://www.learnpytorch.io/03_pytorch_computer_vision/

### 0. Computer vision libraries in Pytorch
* 'torchvision' - base domain library for computer vision - https://docs.pytorch.org/vision/main/index.html
* 'torchvision.datasets' - get datasets and data loading functions for computer vision
* 'torchvision.models' - get pretrained computer vision models that you can apply for your problems
* 'torchvision.transforms' - functions for mainpulating your vision data images
* 'torch.utils.data.Dataset' - Base dataset class for pytorch
* 'torch.utils.data.DataLoader' - Creates a python iterable over a dataset

In [ ]:
#import pytorch
import torch
from torch import nn

# import torchvision
import torchvision
from torchvision import datasets
from torchvision import transforms
from torchvision.transforms import ToTensor

# visualize
import matplotlib.pyplot as plt

# check versions
print(torch.__version__)
print(torchvision.__version__)

## 1. Getting a computer vision dataset

FashionMNIST dataset will be used from torchvision.datasets - https://docs.pytorch.org/vision/main/generated/torchvision.datasets.FashionMNIST.html#torchvision.datasets.FashionMNIST


In [ ]:
# setup training data
train_data = datasets.FashionMNIST(
    root="data", # where to download data to?
    train=True, # do we want the training dataset?
    download=True, # do we want to download yes/no?
    transform=torchvision.transforms.ToTensor(),
    target_transform=None # how do we want to transform the labels/targets?

)
# setup test data
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

In [ ]:
len(train_data), len(test_data)

In [ ]:
# see the first trianing example
image, label = train_data[0]
image, label

In [ ]:
image.shape

In [ ]:
len(train_data.data), len(train_data.targets), len(test_data.data), len(test_data.targets)

In [ ]:
class_names= train_data.classes
class_names

In [ ]:
class_to_idx = train_data.class_to_idx
class_to_idx

In [ ]:
train_data.targets

In [ ]:
# check the shape
print(f"image shape: {image.shape} -> [color_channels, height, width]")
print(f"image label: {class_names[label]}")

### 1.2. Visualizing Random samples of data

In [ ]:
train_data

In [ ]:
import matplotlib.pyplot as plt
image, label = train_data[0]
print(f"Image shape: {image.shape}")

plt.imshow(image.squeeze(), cmap="gray")
plt.title(class_names[label]);
plt.axis(False);

In [ ]:
# plot more images
torch.manual_seed(42)
fig = plt.figure(figsize=(9,9))
rows, cols = 4, 4
for i in range(1, rows*cols+1):
  random_idx = torch.randint(0, len(train_data), size=[1]).item()
  img, label = train_data[random_idx]
  fig.add_subplot(rows, cols, i)
  plt.imshow(img.squeeze(), cmap="gray")
  plt.title(class_names[label])
  plt.axis(False);


In [ ]:
train_data, test_data

## 2. Prepare DataLoader

Right now, out data is in the form of pytorch dataset.
DataLoader turns our dataset into a python iterable
Specifically, we want to turn our data into batches or mini-batches.

1. It is more computationally efficient, as in, your computing hardware may not be able to look (store in memory) at 60000 images in one hit. So we break it down to 32 images at a time (batch size of 32)
2. It gives our neural network more chances to update its gradients per epoch

Andrew ng batch mini size lecure - https://www.youtube.com/watch?v=ed4whd9B-xw

In [ ]:
from torch.utils.data import DataLoader # https://docs.pytorch.org/docs/stable/data.html

# setup the batch size hyperparameter
BATCH_SIZE = 32

# turn dataset into iterables (batches)
train_dataloader = DataLoader(dataset=train_data, # dataset to turn into iterable
                              batch_size=BATCH_SIZE, # how many samples per batch?
                              shuffle=True) # shuffle data every epoch?
test_dataloader = DataLoader(dataset=test_data,
                             batch_size=BATCH_SIZE,
                             shuffle=False)
train_dataloader, test_dataloader

In [ ]:
# Let's check out what we've created
print(f"Dataloaders: {train_dataloader, test_dataloader}")
print(f"Lenght of the train dataloader: {len(train_dataloader)} batches of {BATCH_SIZE}...")
print(f"Lenght of the train dataloader: {len(test_dataloader)} batches of {BATCH_SIZE}...")

In [ ]:
# check out what's inside the training dataloader
train_features_batch, train_labels_batch = next(iter(train_dataloader))
train_features_batch.shape, train_labels_batch.shape

In [ ]:
# show a sample
torch.manual_seed(42)
random_idx = torch.randint(0, len(train_features_batch), size=[1]).item()
img, label = train_features_batch[random_idx], train_labels_batch[random_idx]
plt.imshow(img.squeeze(), cmap="gray")
plt.title(class_names[label])
plt.axis(False)
print(f"Image size: {img.shape}")
print(f"Label: {label.shape}")

## 3. Model 0: Creating a baseline model

When starting to build a series of machine learning modelling experiments, it's best practice to build baseline model

A baseline model is a simple model you will try improve upon with subsequent models/experiments

In other words: start simply add complexity when necessary

In [ ]:
# create a flatten layer
flatten_model = nn.Flatten()

# get a single sample
x = train_features_batch[0]

# flatten the sample
output= flatten_model(x) # perform forward pass

print(f"Shape before flattening: {x.shape} -> [color_channels, height, width]")
print(f"Shape after flattening: {output.shape} -> [color_channels, height* width]")


In [ ]:
output.squeeze().shape

In [ ]:
from torch import nn
class FashionMNISTModelV0(nn.Module):
  def __init__(self,
               input_shape: int,
               hidden_units: int,
               output_shape: int):
    super().__init__()
    self.layer_stack = nn.Sequential(
        nn.Flatten(), # neural networkds like their inputs in vector form
        nn.Linear(in_features=input_shape,
                  out_features=hidden_units),
        nn.Linear(in_features=hidden_units,
                  out_features=output_shape)
    )
  def forward(self, x):
    return self.layer_stack(x)




In [ ]:
torch.manual_seed(42)

# setup model with input parameters
model_0 = FashionMNISTModelV0(
    input_shape=784, # this is 28*28
    hidden_units=10, # how many unit sin the hidden layer
    output_shape=len(class_names) # one for every class
).to("cpu")
model_0

In [ ]:
len(class_names)

In [ ]:
dummy_x = torch.rand([1,1,28,28])
model_0(dummy_x).shape

In [ ]:
model_0.state_dict()

### 3.1 Setup a loss and optimizer and evaluation metrics
* Loss function - since we're working with multi-class data, our loss function will eb 'nn.CrossEntropyLoss()'
* Optimizer - our optimizer 'torch_optim.SDG' (stochastic gradient descent)
* Evaluation metric - since we're working on a classification problem, let's use accuracy as our evaluation metric


In [ ]:
import requests
from pathlib import Path

# download helper functions from learn pytorch repo
if Path("helper_functions.py").is_file():
  print("helper_functions.py already exists, skipping download")
else:
  print("Downloading helper_functions.py")
  request = requests.get("https://raw.githubusercontent.com/mrdbourke/pytorch-deep-learning/refs/heads/main/helper_functions.py")
  with open("helper_functions.py", "wb") as f:
    f.write(request.content)

In [ ]:
# import accuracy metric
from helper_functions import accuracy_fn

# setup loss function and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model_0.parameters(),
                            lr=0.1)


### 3.2 Creating a function to time our experiments

Machine learning is very experimental.

Two of the main things you'll often want to track are:
1. Model's performance (loss and accuracy values etc)
2. How fast it runs



In [ ]:
from timeit import default_timer as timer
def print_train_time(start: float,
                     end: float,
                     device: torch.device=None):
  """Prints difference between start and end time
  """
  total_time = end-start
  print(f"Train time on {device}: {total_time:.3f} seconds")

  return total_time


In [ ]:
start_time = timer()

end_time = timer()
print_train_time(start=start_time,
                 end=end_time,
                 device="cpu")

### 3.3 Creating a training loop and training a model on batches of data

1. Loop through epochs,
2. Loop through training batches, perform training steps, calculate the train loss *per batch*
3. Loop through testing batches, perform testing steps, calculate the test loss per batch
4. Print out what's happening
5. Time it all for fun


In [ ]:
# import tqdm from progress bar
from tqdm.auto import tqdm

# set the seed and start the timer

torch.manual_seed(42)
train_time_start_on_cpu = timer()

# set the number of epochs
epochs =3

# create training and test loop
for epoch in tqdm(range(epochs)):
  print(f"Epoch: {epoch}\n-----")

  ###training
  train_loss=0
  # add a loop to loop through the training batches
  for batch, (X, y) in enumerate(train_dataloader):
    model_0.train()
    # 1. Forward pass
    y_pred=model_0(X)

    # 2. Calculate loss (per batch)
    loss = loss_fn(y_pred, y)
    train_loss += loss # accumulate train loss

    # 3. Optimizer zero grad
    optimizer.zero_grad()

    # 4. Loss backward
    loss.backward()

    # 5. Optimizer step
    optimizer.step()

    # print out
    if batch % 400 == 0:
      print(f"lOoked at {batch*len(X)}/{len(train_dataloader.dataset)} samples.")
  # divide total train loss by length of train dataloader
  train_loss/=len(train_dataloader)

  ### testing
  test_loss, test_acc = 0,0
  model_0.eval()

  with torch.inference_mode():
    for X_test, y_test in test_dataloader:
      # 1. Forward pass
      test_pred = model_0(X_test)

      # 2. Calculate loss (accumulativeluy)
      test_loss += loss_fn(test_pred, y_test).item()

      # 3. Calculate accuracy
      test_acc += accuracy_fn(y_true=y_test,
                              y_pred=test_pred.argmax(dim=1))

    # calculate the test loss average per batch
    test_loss /= len(test_dataloader)

    # calculate the test acc average per batch
    test_acc /= len(test_dataloader)

  print(f"\nTrain loss: {train_loss:.4f} | Test loss: {test_loss:.4f}, Test acc: {test_acc:.4f}")
# calcualte training time
train_time_end_on_cpu=timer()
total_train_time_model_0 = print_train_time(
    start=train_time_start_on_cpu,
    end=train_time_end_on_cpu,
    device=str(next(model_0.parameters()).device))



## 4. Make predictions and get model0 results


In [ ]:
torch.manual_seed(42)
def eval_model(model: torch.nn.Module,
               data_loader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               accuracy_fn):
  """Returns a dictionary containing the results of model predicting on data_loader"""
  loss, acc = 0,0
  model.eval()
  with torch.inference_mode():
    for X, y in tqdm(data_loader):
      # make predictions
      y_pred = model(X)

      # accumulate the loss and acc values per batch
      loss += loss_fn(y_pred, y)
      acc += accuracy_fn(y_true=y,
                         y_pred = y_pred.argmax(dim=1))
    # scalre loss and acc to find the avg loss/acc per batch
    loss /= len(data_loader)
    acc /= len(data_loader)
  return {"model_name": model.__class__.__name__, # only works when model was created with class
          "model_loss": loss.item(),
          "model_acc": acc
          }

# calculate model 0 results on test dataset
model_0_results = eval_model(model=model_0,
                             data_loader=test_dataloader,
                             loss_fn = loss_fn,
                             accuracy_fn=accuracy_fn)

model_0_results


In [ ]:
model_0_results = eval_model(model=model_0,
                             data_loader=test_dataloader,
                             loss_fn = loss_fn,
                             accuracy_fn=accuracy_fn
                             )

model_0_results

In [ ]:
!nvidia-smi

## 5. Setup device agnostic-code (for using a GPU if there is one)

In [ ]:

import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
device

## 6. Model 1: Create non-linear model

We learned the non linear model in model 2.

In [ ]:
from torch.nn.modules.linear import Linear
import torch.nn as nn
# create a model

class FashionMNISTModelV1(nn.Module):
  def __init__(self, input_shape: int,
               hidden_units: int,
               output_shape: int):
    super().__init__()
    self.layer_stack = nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features=input_shape, out_features=hidden_units),
        nn.ReLU(),
        nn.Linear(in_features=hidden_units,out_features=output_shape),
        nn.ReLU()

    )
  def forward(self, x: torch.Tensor):
    return self.layer_stack(x)





In [ ]:
# create an isntance of model_1
torch.manual_seed(42)

model_1 = FashionMNISTModelV1(
    input_shape=784, # this 28*28
    hidden_units=10,
    output_shape=len(class_names)
).to(device)

next(model_1.parameters()).device

### 6.1 Setup loss, optimizer and evaluation metrics


In [ ]:
from helper_functions import accuracy_fn
# create loss function and optimizer

loss_fn = nn.CrossEntropyLoss() # measure how wrong our mdoel is
optimizer = torch.optim.SGD(params=model_1.parameters(), # tries to update our mdoel;s parameters to reduce the loss
                            lr = 0.1)


### 6.2 Functionizing training and evaluation loop

Let's create a function for:
 * training loop : train_step()
 * test loop : test_step()

In [ ]:
def train_step(data_loader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               model: torch.nn.Module,
               accuracy_fn,
               device: torch.device=device
               ):

  """Performs a training with model trying to learn on data_loader"""

  train_loss, train_acc = 0,0

  # put model into trianing mode
  model.train()

  for batch, (X, y) in enumerate(train_dataloader):
    # put data on target device
    X, y = X.to(device), y.to(device)
    # 1. Forward pass
    y_pred = model(X)

    # 2. Calculate loss (per batch)
    loss = loss_fn(y_pred, y)
    train_loss += loss # accumulatively add up the loss per epoch
    train_acc += accuracy_fn(y_true=y,
                             y_pred=y_pred.argmax(dim=1)) # go from logits to prediction label
    # 3. Optimizer zero grad
    optimizer.zero_grad()

    # 4. Loss backward
    loss.backward()

    # 5. Optimizer step
    optimizer.step()

    # Print out how many samples have been seen
    if batch % 400 == 0:
        print(f"Looked at {batch * len(X)}/{len(train_dataloader.dataset)} samples")

  # Divide total train loss by length of train dataloader (average loss per batch per epoch)
  train_loss /= len(data_loader)
  train_acc /= len(data_loader)
  print(f"Train loss: {train_loss:.4f} | Train acc: {train_acc:.2f}%")





In [ ]:
def test_step(model: torch.nn.Module,
              loss_fn: torch.nn.Module,
              data_loader: torch.utils.data.DataLoader,
              accuracy_fn,
              device: torch.device=device):
  """Performs a testing loop step on model going over data_loader."""
  test_loss, test_acc = 0, 0
  # put the model in eval mode
  model.eval()

  # turn on inference mode context manager
  with torch.inference_mode():
    for X, y in tqdm(data_loader):
      # send the data to the target device
      X, y = X.to(device), y.to(device)

      # 1. Forward pass
      test_pred = model(X)

      # 2. Calculate loss (accumulatively)
      test_loss += loss_fn(test_pred, y) # accumulatively add up the loss per epoch

      # 3. Calculate accuracy (preds need to be same as y_true)
      test_acc += accuracy_fn(y_true=y,
                              y_pred=test_pred.argmax(dim=1))

    # Calculations on test metrics need to happen inside torch.inference_mode()
    # Divide total test loss by length of test dataloader (per batch)
    test_loss /= len(data_loader)

    # Divide total accuracy by length of test dataloader (per batch)
    test_acc /= len(data_loader)

    ## Print out what's happening
    print(f"\nTrain loss: {train_loss:.5f} | Test loss: {test_loss:.5f}, Test acc: {test_acc:.2f}%\n")



In [ ]:
from timeit import default_timer as timer
# set the random seed
torch.manual_seed(42)
train_time_start_on_gpu = timer()

epochs = 3

for epoch in tqdm(range(epochs)):
  print(f"Epoch: {epoch}\n--------")
  train_step(data_loader=train_dataloader,
             loss_fn = loss_fn,
             optimizer=optimizer,
             model=model_1,
             accuracy_fn=accuracy_fn,
             device=device
             )
  test_step(model=model_1,
            loss_fn=loss_fn,
            data_loader=test_dataloader,
            accuracy_fn=accuracy_fn,
            device=device)

  train_time_end_on_gpu = timer()
  total_train_time_model_1 = print_train_time(start=train_time_start_on_gpu,
                                              end=train_time_end_on_gpu,
                                              device=device)

In [ ]:
model_0_results

> **Note** Sometimes, depending on your data/hardware you might find that your model trains faster on cpu and gpu.
>
> Why is this?
> 1. It could be that the overhead for copying data/model to and from the pug outweghts the compute benefits offered by the gpu
> 2. The hardware you're using has a better cpu in terms compute capability than the gpu

Read more here: https://horace.io/brrr_intro.html

In [ ]:
torch.manual_seed(42)
def eval_model(model: torch.nn.Module,
               data_loader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               accuracy_fn,
               device=device):
  """Returns a dictionary containing the results of model predicting on data_loader"""
  loss, acc = 0,0
  model.eval()
  with torch.inference_mode():
    for X, y in tqdm(data_loader):
      # make our data device agnostic
      X, y = X.to(device), y.to(device)
      # make predictions
      y_pred = model(X)

      # accumulate the loss and acc values per batch
      loss += loss_fn(y_pred, y)
      acc += accuracy_fn(y_true=y,
                         y_pred = y_pred.argmax(dim=1))
    # scalre loss and acc to find the avg loss/acc per batch
    loss /= len(data_loader)
    acc /= len(data_loader)
  return {"model_name": model.__class__.__name__, # only works when model was created with class
          "model_loss": loss.item(),
          "model_acc": acc
          }




In [ ]:
# get mode_1 results dictionary
model_1_results = eval_model(model=model_1,
                             data_loader=test_dataloader,
                             accuracy_fn=accuracy_fn,
                             loss_fn=loss_fn,
                             device=device)
model_1_results

In [ ]:
model_0_results

## Model 2: Building a convolutional neural network

CNN's are also know as ConvNets.

CNN's are known for their capabilities to find patterns in visual data

cnn explainer: https://poloclub.github.io/cnn-explainer/

In [ ]:
from torch.nn.modules.pooling import MaxPool2d
from torch.nn.modules.conv import Conv2d
from torch.nn.modules import padding
# create a convolutional neural network
class FashionMNISTModelV2(nn.Module):
  """
  Model architecture that replicates the TinyVGG
  model from CNN explainer website
  """
  def __init__(self,input_shape: int, hidden_units: int, output_shape: int):
    super().__init__()
    # create
    self.conv_block_1 = nn.Sequential(
        nn.Conv2d(in_channels=input_shape,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=1), # values we can set ourselves in our NN's are called hyperparameters
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2)
        )
    self.conv_block_2 = nn.Sequential(
        nn.Conv2d(in_channels=hidden_units,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=1),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2)
    )
    self.classifier = nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features=hidden_units*7*7,
                  out_features=output_shape)
    )
  def forward(self, x):
    x = self.conv_block_1(x)
    # print(f"Output shape of conv_block_1 : {x.shape}")
    x = self.conv_block_2(x)
    #  print(f"Output shape of conv_block_2 : {x.shape}")
    x = self.classifier(x)
    # print(f"Output shape of classifier : {x.shape}")
    return x


In [ ]:
torch.manual_seed(42)
model_2 = FashionMNISTModelV2(input_shape=1,
                              hidden_units=10,
                              output_shape=len(class_names)).to(device)
model_2

### 7.2 Stepping thtough 'nn.Conv2d()'
See the documentation here - https://docs.pytorch.org/docs/stable/generated/torch.nn.Conv2d.html

In [ ]:
torch.manual_seed(42)

# create a batch of images
images = torch.randn(size=(32,3,64,64))
test_image = images[0]

print(f"Image batch shape: {image.shape}")
print(f"Singe image shape: {test_image.shape}")


In [ ]:
torch.manual_seed(42)
# create a single conv2d layer
conv_layer=nn.Conv2d(in_channels=3,
                     out_channels=10,
                     kernel_size=3,
                     stride=1,
                     padding=1)

# pass the data through the convolutional layer
conv_output = conv_layer(test_image)
conv_output.shape

### 7.2 Stepping through nn.MaxPool2d()

Documentation https://docs.pytorch.org/docs/stable/generated/torch.nn.MaxPool2d.html

In [ ]:
# print out original iamge shape without unsqueezed simension
print(f"Test image original shape: {test_image.shape}")
print(f"Test imagw with unsqueezed dimension: {test_image.unsqueeze(0).shape}")

# create a sample nn.MaxPool2d
max_pool_layer = nn.MaxPool2d(kernel_size=2)

# pass data through just the conv_layer
test_image_through_conv = conv_layer(test_image.unsqueeze(dim=0))
print(f"Shape after going through conv_layer(): {test_image_through_conv.shape}")

# pass data through the max pool layer
test_image_through_conv_and_max_pool = max_pool_layer(test_image_through_conv)
print(f"Shape after going through conv and maxpool(): {test_image_through_conv_and_max_pool.shape}")

In [ ]:
torch.manual_seed(42)

# create a random tensor with a silimar number of dimensions to our images
random_tensor = torch.randn(size=(1,1,2,2))
print(f"\nRandom tensor: \n {random_tensor}")
print(f"Random tensor shape: {random_tensor.shape}")
# create a maxpooling layer
max_pool_layer = nn.MaxPool2d(kernel_size=2)

# pass the random tensor through the max pool layer
max_pool_tensor = max_pool_layer(random_tensor)
print(f"\nMax pool tensor: \n {max_pool_tensor}")
print(f"Max pool tensor shape: {max_pool_tensor.shape}")


In [ ]:
rand_image_tensor = torch.randn(size=(1,28,28))
rand_image_tensor.shape

In [ ]:
model_2(rand_image_tensor.unsqueeze(0).to(device))

### 7.3 Setup a loss function and optimizer for model 3


In [ ]:
# create the loss funciton
from helper_functions import accuracy_fn

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model_2.parameters(),
                            lr=0.1)

### 7.4 Training and testing model_2 using our training and test fucntions


In [ ]:
torch.manual_seed(42)

# measure time
from timeit import default_timer as timer
train_time_start_model_2_gpu = timer()

# train and test model
epochs = 3
for epoch in tqdm(range(epochs)):
  print(f"Epoch: {epoch}\n------")
  train_step(data_loader=train_dataloader,
             model = model_2,
             loss_fn= loss_fn,
             optimizer=optimizer,
             accuracy_fn=accuracy_fn,
             device=device)
  test_step(model=model_2,
            loss_fn=loss_fn,
            data_loader=test_dataloader,
            accuracy_fn=accuracy_fn,
            device=device)

train_time_end_model_2_gpu = timer()
total_train_time_model_2_gpu  = print_train_time(end = train_time_end_model_2_gpu,
                                                 start = train_time_start_model_2_gpu,
                                                 device=device)



In [ ]:
model_2_results = eval_model(
    model=model_2,
    data_loader=test_dataloader,
    loss_fn=loss_fn,
    accuracy_fn=accuracy_fn,
    device=device
)
model_2_results


## 8. Compare model results and training time.


In [ ]:
import pandas as pd
compare_results = pd.DataFrame([model_0_results,
                                model_1_results,
                                model_2_results])
compare_results

In [ ]:
# add training time to results comparison
compare_results["training_time"] = [total_train_time_model_0,
                                    total_train_time_model_1,
                                    total_train_time_model_2_gpu]

compare_results

In [ ]:
# visualize our model results
compare_results.set_index("model_name")["model_acc"].plot(kind="barh")
plt.xlabel("accuracy (%)")
plt.ylabel("model")

## 9. Make and evaluate random predictions with best model



In [ ]:
def make_predictions(model: torch.nn.Module,
                     data: list,
                     device: torch.device = device):
  pred_probs = []
  model.to(device)
  model.eval()
  with torch.inference_mode():
    for sample in data:
      # prepare the sample (add a batch dimension and pass to target device)
      sample = torch.unsqueeze(sample, dim=0).to(device)

      # forward pass (mdoel output raw logits)
      pred_logit = model(sample)

      # get prediction prob logits->prediction prob
      pred_prob = torch.softmax(pred_logit.squeeze(), dim=0)

      # get pred_prob off the gpu for further calculations
      pred_probs.append(pred_prob.cpu())
  # stack the pred_probs to turn list into a tensor
  return torch.stack(pred_probs)


In [ ]:
import random
# random.seed(42)
test_samples=[]
test_labels=[]
for sample, label in random.sample(list(test_data), k=9):
  test_samples.append(sample)
  test_labels.append(label)

# view the first sample shape
test_samples[0].shape

In [ ]:
plt.imshow(test_samples[0].squeeze(), cmap="gray")
plt.title(class_names[test_labels[0]])

In [ ]:
# make predictions
pred_probs = make_predictions(model=model_2,
                              data=test_samples)

# view first two prediction probabilities
pred_probs[:2]

In [ ]:
# convert to prediction prob to labels
pred_classes = pred_probs.argmax(dim=1)
pred_classes

In [ ]:
test_labels

In [ ]:
# plot predictions
plt.figure(figsize=(9,9))
nrows = 3
ncols = 3
for i, sample in enumerate(test_samples):
  # create subplot
  plt.subplot(nrows,ncols,i+1)

  # plot the target image
  plt.imshow(sample.squeeze(), cmap="gray")

  # find the prediction (in text form, e.g sandal)
  pred_label = class_names[pred_classes[i]]

  # get the truth label (in text form)
  truth_label = class_names[test_labels[i]]

  # create a title for the plot
  titel_text = f"Pred: {pred_label} | Truth: {truth_label}"

  # check for equality between pred and truth and change color of title text
  if pred_label == truth_label:
    plt.title(titel_text, fontsize=10, c="g") # green text if prediction same as truth

  else:
    plt.title(titel_text, fontsize=10, c="r")
  plt.axis(False);

## 10. Making a confusion matrix for further prediction evaluation

Confustion matrix is a fantastic way of evaluating your classification models visually:
https://lightning.ai/docs/torchmetrics/stable/classification/confusion_matrix.html

1. Make predictions with our trained model on the test dataset
2. Make a confusion matrix 'torchmetrics.ConfusionMatrix' https://docs.pytorch.org/ignite/generated/ignite.metrics.confusion_matrix.ConfusionMatrix.html
3. Plot the confusion matrix using mixtend.plotting.plot_confusion_matrix()  
https://rasbt.github.io/mlxtend/user_guide/plotting/plot_confusion_matrix/

In [ ]:
# import tqdm.audo
from tqdm.auto import tqdm

# 1. make predictions with trained model
y_preds = []
model_2.eval()
with torch.inference_mode():
  for X, y in tqdm(test_dataloader, desc="Making predictions..."):
    # send the data and targets to target device
    X, y = X.to(device), y.to(device)
    # do the forward pass
    y_logit = model_2(X)
    # turn predictions from logits -> prediction probs -> prediction label
    y_pred = torch.softmax(y_logit.squeeze(), dim=0).argmax(dim=1)
    # put prediction on CPU for evaluation
    y_preds.append(y_pred.cpu())
# concatenate list of predictions into a tensor
# print(y_preds)
y_pred_tensor = torch.cat(y_preds)
y_pred_tensor[:10]

In [ ]:
len(y_pred_tensor)

In [ ]:
# see if requried packages are installed and i fnot, install them..
try:
  import torchmetrics, mlxtend
  print(f"mlxtend version: {mlxtend.__version__}")
  assert int(mlxtend.__version__.split(".")[1] >= 19, "mlxtend version should be 0.19.0 or higher")
except:
  !pip install -q torchmetrics -U mlxtend
  import torchmetrics, mlxtend
  print(f"mlxtend version: {mlxtend.__version__}")

In [ ]:
import mlxtend
print(mlxtend.__version__)

In [ ]:

from torchmetrics import ConfusionMatrix
from mlxtend.plotting import plot_confusion_matrix

# 2. setup confusion instance and compare predictions to targets
confmat = ConfusionMatrix(num_classes=len(class_names),
                          task="multiclass")
confmat_tensor = confmat(preds = y_pred_tensor,
                         target=test_data.targets)
# 3. plot the confusion matrix
fig, ax = plot_confusion_matrix(
    conf_mat = confmat_tensor.numpy(), # matplotlib likes working with numpy
    class_names=class_names,
    figsize=(10,7)
)

## 11. Save and reload your trained model

In [ ]:
from pathlib import Path

# create model directory path
MODEL_PATH = Path("models")
MODEL_PATH.mkdir(parents=True,
                 exist_ok=True)

# create mdoel save
MODEL_NAME = "03_pytorch_computer_vision_mode_2.pth"
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME

# save the mdoel state dict
print(f"Saving model to: {MODEL_SAVE_PATH}")
torch.save(obj=model_2.state_dict(),
           f=MODEL_SAVE_PATH)

In [ ]:
from pickle import load
# create a new instance
torch.manual_seed(42)
loaded_model_2 = FashionMNISTModelV2(input_shape=1,
                                     hidden_units=10,
                                     output_shape=len(class_names))

# load in the save state_dict()
loaded_model_2.load_state_dict(torch.load(f=MODEL_SAVE_PATH))

# send the model to the target device
loaded_model_2.to(device)

In [ ]:
model_2_results

In [ ]:
# evaluate loaded model
torch.manual_seed(42)

loaded_model_2_results = eval_model(
    model=loaded_model_2,
    data_loader=test_dataloader,
    loss_fn=loss_fn,
    accuracy_fn=accuracy_fn
)

loaded_model_2_results

In [ ]:
# check if model results are close to each other
torch.isclose(torch.tensor(model_2_results["model_loss"]),
              torch.tensor(loaded_model_2_results["model_loss"]),
              atol=1e-02
)